# Laydown Planner - Optimization
Optimizes object placement using 2D bin packing with category-based nesting and stacking.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import ezdxf
import os

print('All imports successful!')

## 2. Load Data

In [ ]:
objects_df = pd.read_csv('../data/sample_objects.csv')

print(f'Loaded {len(objects_df)} object types')
print(f'\nCategories:')
print(objects_df['category'].value_counts())
print(f'\nStackable objects:')
print(objects_df[objects_df['stackable'] == True][['name', 'stackable', 'max_stack_height']])

## 3. Define 2D Bin Packing Classes

In [ ]:
class Rectangle:
    def __init__(self, width, length, obj_id, obj_name, category, weight, stackable, max_stack):
        self.width = width
        self.length = length
        self.obj_id = obj_id
        self.obj_name = obj_name
        self.category = category
        self.weight = weight
        self.stackable = stackable
        self.max_stack = max_stack
        self.rotated = False
        self.x = 0
        self.y = 0
        self.stack_height = 1
    
    def get_width(self):
        return self.length if self.rotated else self.width
    
    def get_length(self):
        return self.width if self.rotated else self.length
    
    def rotate(self):
        self.rotated = not self.rotated
    
    def area(self):
        return self.get_width() * self.get_length()

class Bin:
    def __init__(self, width, length, bin_id):
        self.width = width
        self.length = length
        self.bin_id = bin_id
        self.rectangles = []
    
    def can_fit(self, rect):
        for existing in self.rectangles:
            if self._overlaps(rect, existing):
                return False
        
        if rect.x + rect.get_width() > self.width:
            return False
        if rect.y + rect.get_length() > self.length:
            return False
        
        return True
    
    def _overlaps(self, rect1, rect2):
        return not (rect1.x + rect1.get_width() <= rect2.x or 
                   rect1.x >= rect2.x + rect2.get_width() or
                   rect1.y + rect1.get_length() <= rect2.y or 
                   rect1.y >= rect2.y + rect2.get_length())
    
    def add_rectangle(self, rect):
        if self.can_fit(rect):
            self.rectangles.append(rect)
            return True
        return False
    
    def find_position(self, rect):
        positions = [(0, 0)]
        
        for existing in self.rectangles:
            positions.append((existing.x + existing.get_width(), existing.y))
            positions.append((existing.x, existing.y + existing.get_length()))
        
        for x, y in sorted(positions):
            rect.x = x
            rect.y = y
            if self.can_fit(rect):
                return True
            
            rect.rotate()
            if self.can_fit(rect):
                return True
            rect.rotate()
        
        return False

print('✓ Packing classes defined')

## 4. Implement Packing Algorithm

In [ ]:
def pack_objects(objects_df, bin_width=100, bin_length=100):
    categories = objects_df['category'].unique()
    all_bins = []
    
    for category in categories:
        category_df = objects_df[objects_df['category'] == category]
        
        rects = []
        for _, row in category_df.iterrows():
            rect = Rectangle(
                width=row['width'],
                length=row['length'],
                obj_id=row['object_id'],
                obj_name=row['name'],
                category=row['category'],
                weight=row['weight'],
                stackable=row['stackable'],
                max_stack=row['max_stack_height']
            )
            rects.append(rect)
        
        rects.sort(key=lambda r: r.area(), reverse=True)
        
        bins = []
        for rect in rects:
            placed = False
            
            if rect.stackable:
                for bin_obj in bins:
                    for existing in bin_obj.rectangles:
                        if (existing.obj_id == rect.obj_id and 
                            existing.stack_height < existing.max_stack):
                            rect.x = existing.x
                            rect.y = existing.y
                            existing.stack_height += 1
                            bin_obj.rectangles.append(rect)
                            placed = True
                            break
                    if placed:
                        break
            
            if not placed:
                for bin_obj in bins:
                    if bin_obj.find_position(rect):
                        bin_obj.add_rectangle(rect)
                        placed = True
                        break
            
            if not placed:
                new_bin = Bin(bin_width, bin_length, len(bins))
                if new_bin.find_position(rect):
                    new_bin.add_rectangle(rect)
        
        all_bins.extend(bins)
    
    return all_bins

print('✓ Packing algorithm defined')

## 5. Generate Optimized Laydown Plan

In [ ]:
bins = pack_objects(objects_df, bin_width=100, bin_length=100)

print(f'\n✓ Packing complete')
print(f'Total bins needed: {len(bins)}')
print(f'Total objects packed: {sum(len(bin.rectangles) for bin in bins)}')

for i, bin_obj in enumerate(bins):
    total_area = sum(rect.area() for rect in bin_obj.rectangles)
    utilization = (total_area / (bin_obj.width * bin_obj.length)) * 100
    print(f'\nBin {i}: {len(bin_obj.rectangles)} objects, {utilization:.1f}% utilization')

## 6. Generate DXF from Optimized Plan

In [ ]:
doc = ezdxf.new()
msp = doc.modelspace()

y_offset = 0
for bin_idx, bin_obj in enumerate(bins):
    bin_x = 0
    bin_y = y_offset
    
    msp.add_lwpolyline([
        (bin_x, bin_y), 
        (bin_x + bin_obj.width, bin_y), 
        (bin_x + bin_obj.width, bin_y + bin_obj.length), 
        (bin_x, bin_y + bin_obj.length), 
        (bin_x, bin_y)
    ], close=True)
    
    stack_offset = {}
    for rect in bin_obj.rectangles:
        key = (rect.x, rect.y)
        if key not in stack_offset:
            stack_offset[key] = 0
        offset = stack_offset[key]
        stack_offset[key] += 0.5
        
        x = bin_x + rect.x + offset
        y = bin_y + rect.y + offset
        w = rect.get_width()
        l = rect.get_length()
        
        msp.add_lwpolyline([
            (x, y), 
            (x + w, y), 
            (x + w, y + l), 
            (x, y + l), 
            (x, y)
        ], close=True)
        
        label = f"{rect.obj_name} {rect.weight}kg"
        if rect.stack_height > 1:
            label += f" (x{rect.stack_height})"
        
        msp.add_text(label, dxfattribs={
            'insert': (x + w/2, y + l/2),
            'height': 1,
            'halign': 1,
        })
    
    y_offset += bin_obj.length + 5

os.makedirs('../data', exist_ok=True)
doc.saveas('../data/optimized_laydown.dxf')

print('✓ DXF generated: data/optimized_laydown.dxf')

## 7. Summary

In [ ]:
print('
=== OPTIMIZATION SUMMARY ===')
print(f'Total objects: {len(objects_df)}')
print(f'Total bins created: {len(bins)}')
print(f'\nBin Details:')
for i, bin_obj in enumerate(bins):
    stacked = sum(1 for rect in bin_obj.rectangles if rect.stack_height > 1)
    print(f'  Bin {i}: {len(bin_obj.rectangles)} items, {stacked} stacks')
print('
✓ Optimization complete!')